# imports

In [6]:
import pandas as pd
import numpy as np
from itables import show

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector


# prepare data

In [5]:
show(df.head())

Loading ITables v2.5.2 from the internet... (need help?)


In [ ]:
# load
df = pd.read_csv('../fraud_encoded.csv')

# prepare 
X = df.drop(columns=['is_fraud'])
y = df['is_fraud']

# split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    shuffle=True,
                                                    random_state=42)

# scale
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_train_scaled[['amt', 'city_pop']] = scaler.fit_transform(X_train[['amt', 'city_pop']])
X_test_scaled = X_test.copy()
X_test_scaled[['amt', 'city_pop']] = scaler.transform(X_test[['amt', 'city_pop']])


ValueError: Expected a 1D array, got an array with shape (444575, 2)